BAGGING(BOOTSTRAP + AGGREGATION)

Download heart disease dataset heart.csv in Exercise folder and do following, (credits of dataset: https://www.kaggle.com/fedesoriano/heart-failure-prediction)

1.Load heart disease dataset in pandas dataframe

2.Remove outliers using Z score. Usual guideline is to remove anything that has Z score > 3 formula or Z score < -3

3.Convert text columns to numbers using label encoding and one hot encoding

4.Apply scaling

5.Build a classification model using support vector machine. Use standalone model as well as Bagging model and check if you see any difference in the performance.

6.Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance

7.Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why. Use internet to figure out in what conditions bagging works the best.

In [316]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

df=pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


Outlier Detection and its removal 
Z= (X(i)-X(mean))/(standard deviation of X)
if Z>3 or Z<-3 then remove this data as it's an outlier

In [317]:
from scipy.stats import zscore
# Calculate Z-scores for all numerical columns
z_scores = df.select_dtypes(include=['float64', 'int64']).apply(zscore)

# Set a threshold for Z-score (e.g., ±3)
threshold = 3

# Filter rows where all Z-scores are within the threshold
df_no_outliers = df[(z_scores < threshold).all(axis=1)]

print("Original DataFrame shape:", df.shape)
print("DataFrame shape after removing outliers:", df_no_outliers.shape)
df_no_outliers['RestingECG'].unique()

Original DataFrame shape: (918, 12)
DataFrame shape after removing outliers: (902, 12)


array(['Normal', 'ST', 'LVH'], dtype=object)

LABEL ENCODING AND HOT ENCODING

In [318]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le=LabelEncoder()
df_no_outliers['Sex']=le.fit_transform(df_no_outliers['Sex'])
df_no_outliers['ExerciseAngina']=le.fit_transform(df_no_outliers['ExerciseAngina'])
df_no_outliers['ChestPainType']=le.fit_transform(df_no_outliers['ChestPainType'])
df_no_outliers['RestingECG']=le.fit_transform(df_no_outliers['RestingECG'])
df_no_outliers['ST_Slope']=le.fit_transform(df_no_outliers['ST_Slope'])
df_no_outliers

C:\Users\HP USER\AppData\Local\Temp\ipykernel_13800\2503968899.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers['Sex']=le.fit_transform(df_no_outliers['Sex'])
C:\Users\HP USER\AppData\Local\Temp\ipykernel_13800\2503968899.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_outliers['ExerciseAngina']=le.fit_transform(df_no_outliers['ExerciseAngina'])
C:\Users\HP USER\AppData\Local\Temp\ipykernel_13800\2503968899.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,140,289,0,1,172,0,0.0,2,0
1,49,0,2,160,180,0,1,156,0,1.0,1,1
2,37,1,1,130,283,0,2,98,0,0.0,2,0
3,48,0,0,138,214,0,1,108,1,1.5,1,1
4,54,1,2,150,195,0,1,122,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,110,264,0,1,132,0,1.2,1,1
914,68,1,0,144,193,1,1,141,0,3.4,1,1
915,57,1,0,130,131,0,1,115,1,1.2,1,1
916,57,0,1,130,236,0,0,174,0,0.0,1,1


In [319]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
scaling=StandardScaler()
X=df_no_outliers.drop('HeartDisease',axis='columns')
X_final=X.values
X_final=scaling.fit_transform(X_final)
Y_final=df_no_outliers['HeartDisease'].values
ct = ColumnTransformer([('Car MOdel', OneHotEncoder(), [0])], remainder = 'passthrough')
X_final=ct.fit_transform(X_final)
X_final=X_final[:,1:]

Train Test and Split

In [320]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_final,Y_final,test_size=0.2)

USING SVM MODEL (WITHOUT BAGGING)

In [321]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
model1=SVC()
model1.fit(X_train,Y_train)
print(model1.score(X_test,Y_test))
cross_validation_score=cross_val_score(model1,X_final,Y_final,cv=5)
print(cross_validation_score.mean())

0.8839779005524862
0.8314180478821364


SVM MODEL (WITH BAGGING)

In [322]:
from sklearn.ensemble import BaggingClassifier

clf=BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.9,
    oob_score=True,
    random_state=0
)

clf.fit(X_train,Y_train)
clf.oob_score_

0.8779472954230236

In [323]:
clf.score(X_test,Y_test)

0.8895027624309392

In [324]:
clf_score_2=cross_val_score(clf,X_final,Y_final,cv=5)
clf_score_2.mean()

0.8336525475751996

Using Bagging in SVM there is no such improvement in the ouput of the test data. As you can see above, using bagging in case of SVM doesn't make much difference in terms of model accuracy. Bagging is effective when we have high variance and instable model such as decision tree. Let's explore how bagging changes the performance for a decision tree classifier.

Decision Tree Classifier (Without Bagging)

In [325]:
from sklearn.tree import DecisionTreeClassifier

model2=DecisionTreeClassifier()
model2.fit(X_train,Y_train)
print(model2.score(X_test,Y_test))
cross_val_score_dec_tree=cross_val_score(model2,X_final,Y_final,cv=5)
print(cross_val_score_dec_tree.mean())

0.8342541436464088
0.7571147943523633


Decision tree (With bagging)

In [326]:
clf_new=BaggingClassifier(
    estimator=model2,
    n_estimators=100,
    max_samples=0.9,
    oob_score=True,
    random_state=0
)

clf_new.fit(X_train,Y_train)
clf_new.oob_score_

0.841886269070735

In [327]:
clf_new.score(X_test,Y_test)

0.850828729281768

In [328]:
cross_val_score_dec_tree_2=cross_val_score(clf_new,X_final,Y_final,cv=5)
cross_val_score_dec_tree_2.mean()

0.8058931860036832

There is almost 5 percent score improvement by using Bagging in Decision Tree Classifier